Reinforce log vizualizer

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_csv("10_rl_corrector_None_stacked_mlp.csv")


sns.kdeplot(x="reward", data=df, fill=True)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

# Load the CSV file
df = pd.read_csv("10_rl_corrector_None_stacked_mlp.csv")

# Convert the reward_list strings into actual Python lists
df['reward_list'] = df['reward_list'].apply(ast.literal_eval)

# Pad all reward lists to the same length with NaNs
max_len = max(len(r) for r in df['reward_list'])
reward_matrix = np.array([
    r + [np.nan] * (max_len - len(r))  # pad with NaNs for consistent shape
    for r in df['reward_list']
])

# Compute mean and standard deviation across runs
mean_rewards = np.nanmean(reward_matrix, axis=0)
std_rewards = np.nanstd(reward_matrix, axis=0)

# Plot mean reward and standard deviation band
plt.figure(figsize=(12, 6))
episodes = np.arange(max_len)
plt.plot(episodes, mean_rewards, label='Mean Reward', color='blue')
plt.fill_between(episodes, mean_rewards - std_rewards, mean_rewards + std_rewards,
                 alpha=0.3, label='±1 Std Dev', color='blue')
plt.title('Mean and Std Dev of Rewards per Episode Across Runs')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast

# Load and prepare the data
df = pd.read_csv('10_rl_corrector_None_stacked_mlp_PPO.csv')
df['reward_list'] = df['reward_list'].apply(ast.literal_eval)

# Convert reward lists to long-form DataFrame
records = []
for run_id, rewards in enumerate(df['reward_list']):
    for episode, reward in enumerate(rewards):
        records.append({'Run': run_id, 'Episode': episode, 'Reward': reward})

long_df = pd.DataFrame.from_records(records)

# Set seaborn style
sns.set_theme(style='whitegrid', palette='viridis', font_scale=1.2)

# Plot with confidence interval (default: 95%)
plt.figure(figsize=(12, 6))
sns.lineplot(data=long_df, x='Episode', y='Reward', errorbar='sd', color='purple')  # Use ci='sd' for ±1 std deviation
plt.title('Reward per Episode with ±1 Std Dev (Seaborn)')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import re


def clean_label(file):
    name = os.path.splitext(os.path.basename(file))[0]
    name = re.sub(r'^10_', '', name)                         # Remove leading "10_"
    name = name.replace('_', ' ')                            # Replace _ with space
    name = name.replace('rl corrector', 'Reinforce')
    name = name.replace('None', 'C0')
             # Replace label
    name = re.sub(r'\b20(\.0)?\b', '', name)                 # Remove 20 or 20.0
    name = re.sub(r'\s+', ' ', name)                         # Remove extra whitespace
    name = re.sub(r'\b5(\.0)?\b', '', name)        
    return name.strip()


# Set up Seaborn style
sns.set_theme(style="whitegrid", font_scale=1.2)

# Find all CSV files in the current directory
csv_files = glob.glob("*.csv")
print("csv_files", csv_files)
# Prepare data for box plot
data = []

for file in csv_files:
    try:
        df = pd.read_csv(file)
        if 'reward' in df.columns:
            label = clean_label(file)
            for value in df['reward']:

                data.append({'File': label, 'Reward': value})
    except Exception as e:
        print(f"Skipping {file} due to error: {e}")

# Convert to DataFrame
plot_df = pd.DataFrame(data)

# Create box plot
plt.figure(figsize=(14, 6))
ax = sns.boxplot(data=plot_df, x='File', y='Reward', palette="viridis")

# Aesthetic tweaks
plt.xticks(rotation=45, ha='right')
plt.title("Distribution of Final Rewards Across CSV Files")
plt.xlabel("corrector")
plt.ylabel("Final Reward")
plt.tight_layout()
plt.grid(True, linestyle='--', alpha=0.5)

# Save and show
# plt.savefig("plots/boxplot_all_runs.png", dpi=300)
plt.show()

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import ast

# Load the CSV file
file_path = '10_rl_corrector_None_stacked_mlp_PPO.csv'
data = pd.read_csv(file_path)

# Define a function to extract numerical values from the string representation of arrays
def extract_first_values_from_string(s):
    # Use regular expression to find all array representations in the string
    array_strings = re.findall(r'array\(\s*\[([-+]?\d*\.\d+|\d+)\s*\]', s)
    # Extract the first numerical value from each array string
    return [float(arr) for arr in array_strings]

# Extract the first values from the 'reward_list' column using the refined method
reward_lists = []
for reward_list in data['reward_list']:
    print(reward_list)
    try:
        # Extract the first numerical values from the string
        values = extract_first_values_from_string(reward_list)
        # first_reward_values.extend(values)
        reward_lists.append(values)
    except (ValueError, SyntaxError):
        # Skip entries with unexpected formats
        continue

# Plot the first reward values extracted from the 'reward_list' column
plt.figure(figsize=(12, 6))
plt.plot(reward_lists, marker='o', linestyle='-')
plt.title('First Reward Values from Each Array in Reward List')
plt.xlabel('Index')
plt.ylabel('Reward')
plt.grid(True)
plt.show()



In [1]:
from classic_rl import rl_corrector
import gymnasium as gym

env = gym.make("MountainCarContinuous-v0")  # default goal_velocity=0

corrector = rl_corrector.PPOSB3Corrector(env, None) 
rlist  = corrector.train(10)



2025-06-27 15:59:34.136422: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-27 15:59:34.162145: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-27 15:59:35.964 | INFO     | classic_rl.rl_corrector:__init__:985 - gamma 0.99, clip_epsilon 0.2, gae_lambda 1, num_episodes 10, k_epochs 80, vf_loss_coef 0.5, action_std 0.6, decay_action_std_rate 0.05, min_action_std 0.1


Device set to : NVIDIA RTX 2000 Ada Generation Laptop GPU
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/jmartinsaquet/.pyenv/versions/env_aiming/lib/python3.11/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 32`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 32
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=32 and n_envs=1)
  warnings.warn(
/home/jmartinsaquet/.pyenv/versions/env_aiming/lib/python3.11/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the tra

TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

In [2]:
from classic_rl import rl_corrector
import gymnasium as gym

env = gym.make("MountainCarContinuous-v0")  # default goal_velocity=0

corrector = rl_corrector.PPOSB3Corrector(env, None) 
rlist  = corrector.train(10)



2025-06-27 15:59:52.367 | INFO     | classic_rl.rl_corrector:__init__:985 - gamma 0.99, clip_epsilon 0.2, gae_lambda 1, num_episodes 10, k_epochs 80, vf_loss_coef 0.5, action_std 0.6, decay_action_std_rate 0.05, min_action_std 0.1


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/jmartinsaquet/.pyenv/versions/env_aiming/lib/python3.11/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 32`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 32
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=32 and n_envs=1)
  warnings.warn(


TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

In [ ]:
rlist

In [5]:
import stable_baselines3 as sb

env = gym.make("MountainCarContinuous-v0")  


mopel = sb.PPO(
    "MlpPolicy",env, verbose=2)

mopel.learn(total_timesteps=100000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 999      |
|    ep_rew_mean     | -53.6    |
| time/              |          |
|    fps             | 1752     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 999         |
|    ep_rew_mean          | -51.3       |
| time/                   |             |
|    fps                  | 1366        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007100623 |
|    clip_fraction        | 0.0271      |
|    clip_range           | 0.2         |
|    entropy_loss  

In [36]:
import torch

state = torch.tensor([[0.0, 0.0]]).to(mopel.device)
mopel.policy(state, deterministic=True)

state.shape

torch.Size([1, 2])